In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.colors as mcolors
import contextily as ctx 


In [16]:
geo_regions = pd.read_csv("Data\geographical_regions.csv")
animals = pd.read_csv("Data\species_info_annie.csv")
taxonomy = pd.read_csv("traits (1).csv")
env = pd.read_csv("Data\species\environmental_df.csv")

In [37]:
taxonomy

,Species ID,Body symmetry,actual evapotranspiration rate in geographic range (millimeters per month),adult yearly survival (percent),age at eye opening (days),age at maturity (days),amino acid composition of milk,animal population density (individuals per square kilometer),are commensal with,are eaten by,...,vocalization behavior,water dissolved o2 concentration (mL/L),water nitrate concentration (µmol/l),water o2 saturation (percent),water phosphate concentration (µmol/l),water salinity (psu),water silicate concentration (µmol/l),water temperature (degrees celsius),weaning age (days),wet body mass (g)
0,1018152,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"('Asio otus (Linnaeus 1758) (Long-eared Owl)',...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1018724,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"('Uromacer catesbyi (Schlegel 1837)', 'Uromace...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1018728,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1018733,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1018894,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"('Thamnophis sirtalis sirtalis (Linnaeus 1758)',)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,962581,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,963127,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,963761,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,964195,('bilaterally symmetricURI:http://purl.obolibr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
geo_regions['Latitude Min'] = geo_regions['Latitude Range'].apply(lambda x: x[0])
geo_regions['Latitude Max'] = geo_regions['Latitude Range'].apply(lambda x: x[1])
geo_regions['Longitude Min'] = geo_regions['Longitude Range'].apply(lambda x: x[0])
geo_regions['Longitude Max'] = geo_regions['Longitude Range'].apply(lambda x: x[1])

# Define a function to find the matching leaf label for each row in env using vectorized operations
def get_leaf_label(row):
    # Vectorized comparison
    match = geo_regions[
        (geo_regions['Latitude Min'] <= row['Latitude']) & 
        (geo_regions['Latitude Max'] >= row['Latitude']) &
        (geo_regions['Longitude Min'] <= row['Longitude']) & 
        (geo_regions['Longitude Max'] >= row['Longitude'])
    ]
    # If a match is found, return the Leaf Number; otherwise, return None
    if not match.empty:
        return match['Leaf Number'].values[0]  # Return the first match if multiple matches
    else:
        return None

# Apply the function to each row in env to add the Leaf Label
env['Leaf Label'] = env.apply(get_leaf_label, axis=1)

In [53]:
env

,ID,Latitude,Longitude,Cluster,BIO1,BIO5,BI06,BIO12,BIO15
0,31529,-18.286728,143.481247,0,25.830833,36.812252,12.261750,778.0,118.931313
1,31529,-18.286728,143.481247,0,25.830833,36.812252,12.261750,778.0,118.931313
2,31529,-18.286728,143.481247,0,25.830833,36.812252,12.261750,778.0,118.931313
3,31529,-18.286728,143.481247,0,25.830833,36.812252,12.261750,778.0,118.931313
4,31529,-18.286728,143.481247,0,25.830833,36.812252,12.261750,778.0,118.931313
...,...,...,...,...,...,...,...,...,...
271265,145031,12.899637,79.348824,3,28.194969,38.084751,18.804251,1020.0,76.790619
271266,145031,22.905533,84.214233,3,23.526134,37.633999,9.562250,1491.0,122.532074
271267,145031,24.600239,72.730560,3,23.997032,37.051250,9.970500,1013.0,166.995056
271268,145031,18.849600,80.654129,3,26.365208,39.693748,13.544250,1547.0,135.629791


In [41]:
env['Unique Label'] = env.apply(lambda x: f"{x['Leaf Label']}_{x['Cluster']}", axis=1)

In [44]:
df = env[["ID", "Unique Label"]]
df

,ID,Unique Label
0,31529,168.0_0.0
1,31529,168.0_0.0
2,31529,168.0_0.0
3,31529,168.0_0.0
4,31529,168.0_0.0
...,...,...
271265,145031,145.0_3.0
271266,145031,145.0_3.0
271267,145031,145.0_3.0
271268,145031,145.0_3.0


In [9]:
# Count the number of NaN values for each column
nan_counts = taxonomy.isna().sum()

# Convert the column names and their NaN counts into a list of tuples
nan_counts_list = [(col, count) for col, count in nan_counts.items()]

# Print the list of columns with their NaN counts
print("List of columns with their NaN counts:")
print(nan_counts_list)


List of columns with their NaN counts:
[('Species ID', 0), ('Body symmetry', 6), ('actual evapotranspiration rate in geographic range (millimeters per month)', 425), ('adult yearly survival (percent)', 493), ('age at eye opening (days)', 477), ('age at maturity (days)', 456), ('amino acid composition of milk', 493), ('animal population density (individuals per square kilometer)', 455), ('are commensal with', 493), ('are eaten by', 338), ('are host of', 291), ('are killed by', 482), ('are mutualistic with', 479), ('are parasitized by', 310), ('are preyed upon by', 317), ('are vectors for', 486), ('ash content (percent)', 488), ('auditory system', 3), ('basal metabolic rate (watts)', 448), ('behavioral circadian rhythm', 316), ('bill length (mm)', 488), ('body length (mm)', 422), ('body mass (g)', 152), ('body shape', 391), ('body temperature (kelvin)', 458), ('breeding habitat', 488), ('carbohydrate composition of milk', 484), ('cellularity', 3), ('clutch/brood/litter size', 374), ('co-

In [47]:
df = pd.read_csv("Data\combined_taxonomyDF_traitsDF.csv")
df

,id,scientific_name,family,kingdom,phylum,parent,class,genus,Species ID,Body symmetry,...,vocalization behavior,water dissolved o2 concentration (mL/L),water nitrate concentration (µmol/l),water o2 saturation (percent),water phosphate concentration (µmol/l),water salinity (psu),water silicate concentration (µmol/l),water temperature (degrees celsius),weaning age (days),wet body mass (g)
0,458,Aramides saracura,Rallidae,Animalia,Chordata,Aramides,Aves,Aramides,47048109,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,460,Aramides ypecaha,Rallidae,Animalia,Chordata,Aramides,Aves,Aramides,1052901,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,487,Fulica cristata,Rallidae,Animalia,Chordata,Fulica,Aves,Fulica,1050369,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,585,Tetraogallus tibetanus,Phasianidae,Animalia,Chordata,Tetraogallus,Aves,Tetraogallus,311569,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(317.1, 265.0, 371.37)",NaN
4,871,Alectoris barbara,Phasianidae,Animalia,Chordata,Alectoris,Aves,Alectoris,311526,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(35.099999999999994, 36.0)",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1289606,Urile urile,Phalacrocoracidae,Animalia,Chordata,Phalacrocorax,Aves,Phalacrocorax,327294,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(133.76,)",NaN
496,1289647,Chrysuronia versicolor,Trochilidae,Animalia,Chordata,Chrysuronia,Aves,Chrysuronia,45513773,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,1289681,Riccordia ricordii,Trochilidae,Animalia,Chordata,Riccordia,Aves,Riccordia,52571190,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,1289689,Campocolinus coqui,Phasianidae,Animalia,Chordata,Campocolinus,Aves,Campocolinus,45510632,('bilaterally symmetricURI:http://purl.obolibr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
import re

cols = ["id","class","habitat", "conservation status", "ecoregion", "body temperature (kelvin)", 'body mass (g)', "locomotion", "motility", "population trend", "hearing range (Hz)", "reproduction", "social system"]

df = df[cols]

def clean_text(text):
    if pd.isna(text):  # Check for NaN and skip cleaning
        return text
    # Remove everything from 'URI' onward, any parentheses, and single quotes
    cleaned_text = re.sub(r'URI.*', '', str(text)).strip("()").replace("'", "").strip()
    return cleaned_text

# Apply the cleaning function to all columns in the DataFrame
for col in df.columns:
    df[col] = df[col].apply(clean_text)

# Define the mapping dictionary after cleaning
conservation_mapping = {
    "cites appendix ii": "vulnerable (trade)",
    "cites appendix i": "vulnerable (trade)",
    "cites appendix iii": "vulnerable (trade)"
}

#'hearing range (Hz)', 'body temperature (kelvin)', 'body mass (g)'

def calculate_cell_mean(cell):
    if pd.isna(cell):  # If cell is NaN, return it as is
        return cell
    # Split the cell by commas, convert to floats, and calculate the mean
    values = [float(value) for value in str(cell).split(',') if value.strip()]
    return np.mean(values) if values else np.nan

# Apply the function to each cell in the DataFrame
for col in ['hearing range (Hz)', 'body temperature (kelvin)', 'body mass (g)']:
    df[col] = df[col].apply(calculate_cell_mean)

# Display the DataFrame with mean values in each cell
print(df.head())
# Apply the mapping to the 'conservation status' column
df['conservation status'] = df['conservation status'].replace(conservation_mapping)

df

    id class         habitat conservation status  \
0  458  Aves        mountain                 NaN   
1  460  Aves     terrestrial       least concern   
2  487  Aves         montane       least concern   
3  585  Aves  montane forest       least concern   
4  871  Aves         coastal       least concern   

                       ecoregion  body temperature (kelvin)  body mass (g)  \
0  Sulu Archipelago Rain Forests                        NaN         60.750   
1   Alto Paraná Atlantic Forests                        NaN         21.600   
2        Central Andean Dry Puna                        NaN         24.300   
3   Western Zambezian Grasslands                        NaN      40970.666   
4    Mississippi Lowland Forests                     311.35         68.070   

      locomotion         motility population trend  hearing range (Hz)  \
0        running              NaN              NaN              1250.0   
1         flight              NaN          Stable,              3000.0

C:\Users\annie\AppData\Local\Temp\ipykernel_20128\2380083424.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(clean_text)
C:\Users\annie\AppData\Local\Temp\ipykernel_20128\2380083424.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(clean_text)
C:\Users\annie\AppData\Local\Temp\ipykernel_20128\2380083424.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,id,class,habitat,conservation status,ecoregion,body temperature (kelvin),body mass (g),locomotion,motility,population trend,hearing range (Hz),reproduction,social system
0,458,Aves,mountain,NaN,Sulu Archipelago Rain Forests,NaN,60.7500,running,NaN,NaN,1250.0,sexual reproduction,NaN
1,460,Aves,terrestrial,least concern,Alto Paraná Atlantic Forests,NaN,21.6000,flight,NaN,"Stable,",3000.0,NaN,NaN
2,487,Aves,montane,least concern,Central Andean Dry Puna,NaN,24.3000,flight,NaN,"Stable,",3000.0,NaN,NaN
3,585,Aves,montane forest,least concern,Western Zambezian Grasslands,NaN,40970.6660,running,actively mobile,"Decreasing,",NaN,NaN,no harem
4,871,Aves,coastal,least concern,Mississippi Lowland Forests,311.35,68.0700,non-cursorial,actively mobile,"Stable,",NaN,NaN,solitary
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1289606,Aves,swamp,vulnerable (trade),Australian Alps Montane Grasslands,310.15,407.0875,flight,actively mobile,"Unknown,",NaN,NaN,group living
496,1289647,Aves,terrestrial,least concern,NaN,NaN,NaN,flight,NaN,"Stable,",3000.0,NaN,NaN
497,1289681,Aves,NaN,NaN,NaN,NaN,NaN,flight,NaN,NaN,3000.0,NaN,NaN
498,1289689,Aves,fossorial and/or ground dwelling only,vulnerable,Western Short Grasslands,NaN,746.0000,flight,actively mobile,"Decreasing,",3000.0,NaN,NaN


,id,lat,long,locatonCluster,leaf_number
0,31529,-18.286728,143.481250,NaN,338
1,31529,-13.099798,130.783650,NaN,338
2,31529,-13.965274,131.695140,NaN,338
3,31529,-12.853950,132.800500,NaN,338
4,31529,-12.196790,134.279330,NaN,338
...,...,...,...,...,...
272032,145031,33.716885,73.203620,NaN,293
272033,145031,24.600239,72.730560,NaN,293
272034,145031,18.849600,80.654130,NaN,293
272035,145031,21.073837,75.945656,NaN,293
